# DV2 Assignment - ETL I

# 1.0 Objective
- Load, Clean, Transform Data for datasets related to HDB Dwelling Units, Residential Age, Electricity Consumption

# 2.0 Imports

In [1]:
# Data
import pandas as pd
import numpy as np

# Viz
#from matplotlib import pyplot as plt
#import seaborn as sns
#sns.set_theme()  # default theme
# sns.set(style='whitegrid')
# References
# http://seaborn.pydata.org/introduction.html


# 3.0 Dataframes to Clean and Create

###  3.1 Clean and Transform Data: Regions and Planning Areas
output: dfRegions

##### 3.1.1 Load Data

In [2]:
dfRegions = pd.read_csv('Data/EMA/3_6_Ave_Ann_HH_Elec_Cons_by_Plan_Area_Dwell_Type/OutputFile.csv')

print(dfRegions.shape)
print(dfRegions.head(3))

(2175, 5)
   year planning_area          region    dwelling_type  \
0  2019        Bishan  Central Region  1-room / 2-room   
1  2019   Bukit Merah  Central Region  1-room / 2-room   
2  2019   Bukit Timah  Central Region  1-room / 2-room   

  average_monthly_electricity_consumption_kwh  
0                                       142.7  
1                                         158  
2                                       150.3  


##### 3.1.2 Drop columns that are not required

In [3]:
dfRegions = dfRegions.drop(columns=['year','dwelling_type','average_monthly_electricity_consumption_kwh'])
dfRegions.head(3)

,planning_area,region
0,Bishan,Central Region
1,Bukit Merah,Central Region
2,Bukit Timah,Central Region


##### 3.1.3 View the nature of the data

In [4]:
print(dfRegions['planning_area'].unique().shape)
print(dfRegions['planning_area'].unique())
print()
print(dfRegions['region'].unique().shape)
print(dfRegions['region'].unique())

(29,)
['Bishan' 'Bukit Merah' 'Bukit Timah' 'Geylang' 'Kallang' 'Marine Parade'
 'Novena' 'Outram' 'Queenstown' 'Rochor' 'Tanglin' 'Toa Payoh' 'Bedok'
 'Pasir Ris' 'Tampines' 'Ang Mo Kio' 'Hougang' 'Punggol' 'Sengkang'
 'Serangoon' 'Sembawang' 'Woodlands' 'Yishun' 'Bukit Batok'
 'Bukit Panjang' 'Choa Chu Kang' 'Clementi' 'Jurong East' 'Jurong West']

(5,)
['Central Region' 'East Region' 'North East Region' 'North Region'
 'West Region']


##### 3.1.4 Remove the Duplicates and Check

In [5]:
dfRegions.drop_duplicates(inplace=True, ignore_index=True)
dfRegions= dfRegions[['region','planning_area']]  # reorder columns
dfRegions = dfRegions.sort_values(by=['region','planning_area'])
dfRegions

,region,planning_area
0,Central Region,Bishan
1,Central Region,Bukit Merah
2,Central Region,Bukit Timah
3,Central Region,Geylang
4,Central Region,Kallang
5,Central Region,Marine Parade
6,Central Region,Novena
7,Central Region,Outram
8,Central Region,Queenstown
9,Central Region,Rochor


In [6]:
print(dfRegions.shape)
print(dfRegions.dtypes)

(29, 2)
region           object
planning_area    object
dtype: object


##### 3.1.5 Save the output to CSV

In [7]:
dfRegions.to_csv('dfRegions.csv',index=False)

###  3.2 Clean and Transform Data: Dwelling Units in HDB
output: dfDwellUnits

##### 3.2.1 Load DataSet 1 - Number of Dwelling Units

In [8]:
dfDwellUnits = pd.read_csv('Data\Data_GOV_SG\HDB\Resi_Units_under_HDB_Mgt\dwelling_units_under_hdbs_management_by_town_and_flat_type.csv')
display(dfDwellUnits.shape)
display(dfDwellUnits.dtypes,'\n')
display(dfDwellUnits.head(3))

(4238, 5)

financial_year           int64
town_or_estate          object
flat_type               object
sold_or_rental          object
no_of_dwelling_units     int64
dtype: object

'\n'

,financial_year,town_or_estate,flat_type,sold_or_rental,no_of_dwelling_units
0,2008,Ang Mo Kio,1-room,Rental Units,1318
1,2008,Ang Mo Kio,1-room,Sold Units,0
2,2008,Ang Mo Kio,2-room,Rental Units,2860


In [9]:
# check for nulls
dfDwellUnits.isnull().sum()

financial_year          0
town_or_estate          0
flat_type               0
sold_or_rental          0
no_of_dwelling_units    0
dtype: int64

##### 3.2.2 Consolidate 'Rental Units' and 'Sold Units' into One line item, and reorg dataframe

In [10]:
# combine rental and sold units into one feature
dfDwellUnits = dfDwellUnits.pivot(index=['financial_year','town_or_estate','flat_type'], columns='sold_or_rental',
                                  values='no_of_dwelling_units')
dfDwellUnits = pd.DataFrame(dfDwellUnits.to_records())
dfDwellUnits.head(3)

,financial_year,town_or_estate,flat_type,Rental Units,Sold Units
0,2008,Ang Mo Kio,1-room,1318.0,0.0
1,2008,Ang Mo Kio,2-room,2860.0,633.0
2,2008,Ang Mo Kio,3-room,62.0,24437.0


In [11]:
# check for Nulls
dfDwellUnits.isnull().sum()

financial_year      0
town_or_estate      0
flat_type           0
Rental Units      470
Sold Units          0
dtype: int64

In [12]:
# replace Nulls with 0, coz adding a number to a NaN results in a NaN
dfDwellUnits['Rental Units'].fillna(0,inplace=True)

In [13]:
# create a new column to combine both 'Rental Units' and 'sold Units'
dfDwellUnits['Num_DwellUnits'] = dfDwellUnits['Rental Units'] + dfDwellUnits['Sold Units']

# drop unneeded cols
dfDwellUnits.drop(columns=['Rental Units','Sold Units'], axis=1, inplace=True)

# replace null with 0
dfDwellUnits['Num_DwellUnits'].fillna(value=0,inplace=True)

# dfDwellUnits.to_csv('dfDwellUnits.csv', index=False)  

dfDwellUnits.head(3)

,financial_year,town_or_estate,flat_type,Num_DwellUnits
0,2008,Ang Mo Kio,1-room,1318.0
1,2008,Ang Mo Kio,2-room,3493.0
2,2008,Ang Mo Kio,3-room,24499.0


##### 3.2.3 Consolidate 'flat_types' to be consistent with EMA's definitions
since we are taking the electricity consumptions figures (which is the purpose of this assignment) from EMA.

EMA:
- 1-room/2-room 
    - includes studio apartments
- 3-room
- 4-room
- 5-room
    - includes Executive, Masionettes, HUDC
    
Housing and Urban Development Company (HUDC) flats are a type of housing unique to Singapore. They were first built in 1974 to cater to a sandwiched class of Singaporeans who can afford something better than the typical public housing, or HDB flats, but yet still find private housing unaffordable. (https://en.wikipedia.org/wiki/Housing_and_Urban_Development_Company_flats)
HUDC is higher end than 5 room but are not private housing.

In [14]:
# temp cell
# dfDwellUnits[(dfDwellUnits['town_or_estate'] == 'Choa Chu Kang') & (dfDwellUnits['financial_year'] == 2018)
#             & (dfDwellUnits['flat_type'] == '5-room')]

In [15]:
# Reorganising Dwelling Types to be the same as the types used in the 'Electricity Consumption' report 
# for easier querying
dfDwellUnits = dfDwellUnits.pivot(index=['financial_year','town_or_estate'], columns='flat_type',
                                  values='Num_DwellUnits')
dfDwellUnits = pd.DataFrame(dfDwellUnits.to_records())
dfDwellUnits.head(3)

,financial_year,town_or_estate,1-room,2-room,3-room,4-room,5-room,Executive,HUDC,Studio Apartment
0,2008,Ang Mo Kio,1318.0,3493.0,24499.0,13026.0,5252.0,481.0,0.0,0.0
1,2008,Bedok,2250.0,1825.0,22583.0,20247.0,10182.0,2712.0,0.0,130.0
2,2008,Bishan,396.0,0.0,2359.0,9199.0,5395.0,1660.0,358.0,0.0


In [16]:
# are there any NaN? If yes, fill it up with 0 (which is correct in this case), otherwise, 
# in future summing, adding a number to a NaN results in a NaN, which will create issues.
# note: HUDC are no longer being built and there were not many of them in the first place.
# instead of HUDC, the government started building EC (executive condominiums)
dfDwellUnits.isnull().sum()

financial_year        0
town_or_estate        0
1-room                0
2-room                0
3-room                0
4-room                0
5-room                0
Executive             0
HUDC                158
Studio Apartment      0
dtype: int64

In [17]:
dfDwellUnits['HUDC'].fillna(0,inplace=True)

In [18]:
# temp cell
# dfDwellUnits[(dfDwellUnits['town_or_estate'] == 'Choa Chu Kang') & (dfDwellUnits['financial_year'] == 2018)]            

In [19]:
dfDwellUnits['1rm_2rm'] = dfDwellUnits['1-room'] + dfDwellUnits['2-room'] + dfDwellUnits['Studio Apartment']
dfDwellUnits['5rm_Exec'] = dfDwellUnits['5-room'] + dfDwellUnits['Executive'] + dfDwellUnits['HUDC']

dfDwellUnits.drop(['1-room','2-room','Studio Apartment','5-room','Executive','HUDC'],axis=1, inplace=True)
dfDwellUnits.head(3)

,financial_year,town_or_estate,3-room,4-room,1rm_2rm,5rm_Exec
0,2008,Ang Mo Kio,24499.0,13026.0,4811.0,5733.0
1,2008,Bedok,22583.0,20247.0,4205.0,12894.0
2,2008,Bishan,2359.0,9199.0,396.0,7413.0


In [20]:
dfDwellUnits.rename(columns = {    
    "3-room":'3rm',
    "4-room":'4rm'    
}, inplace=True)

dfDwellUnits = dfDwellUnits[['financial_year','town_or_estate','1rm_2rm','3rm','4rm','5rm_Exec']]
dfDwellUnits.head(3)

,financial_year,town_or_estate,1rm_2rm,3rm,4rm,5rm_Exec
0,2008,Ang Mo Kio,4811.0,24499.0,13026.0,5733.0
1,2008,Bedok,4205.0,22583.0,20247.0,12894.0
2,2008,Bishan,396.0,2359.0,9199.0,7413.0


In [21]:
dfDwellUnits = pd.melt(dfDwellUnits, id_vars=['financial_year','town_or_estate'],
        value_vars=['1rm_2rm','3rm','4rm','5rm_Exec'],
        var_name='Dwelling_Type', value_name='num_DwellUnits')

dfDwellUnits.num_DwellUnits = dfDwellUnits.num_DwellUnits.astype('int64')

#dfDwellUnits.to_csv('test.csv', index=False)

dfDwellUnits.head(3)

# dfDwellUnits.to_csv('dfDwellUnits.csv', index=False)  

,financial_year,town_or_estate,Dwelling_Type,num_DwellUnits
0,2008,Ang Mo Kio,1rm_2rm,4811
1,2008,Bedok,1rm_2rm,4205
2,2008,Bishan,1rm_2rm,396


##### 3.2.4 Reconcile 'town_or_estate' vs 'planning_area' 
Is there any difference between 'town_or_estate' and 'planning_area' in the previous dataset?

In [22]:
for item in dfDwellUnits.town_or_estate.unique():
    if item not in dfRegions.planning_area.values:
        print(item)    

Central Area
Kallang/Whampoa
Tengah


dfDwellUnits has 3 items not found in dfRegion.planning_area.

In [23]:
for item in dfRegions.planning_area:
    if item not in dfDwellUnits.town_or_estate.unique():
        print(item)    

Kallang
Novena
Outram
Rochor
Tanglin


dfRegion.planning_area has 5 items not found in dfDwellUnits.<br>

In [24]:
dfRegions[dfRegions.region=='Central Region'].sort_values(by='planning_area')

,region,planning_area
0,Central Region,Bishan
1,Central Region,Bukit Merah
2,Central Region,Bukit Timah
3,Central Region,Geylang
4,Central Region,Kallang
5,Central Region,Marine Parade
6,Central Region,Novena
7,Central Region,Outram
8,Central Region,Queenstown
9,Central Region,Rochor


Of the planning_areas above, dfDwellUnits has all except for 'Novena', 'Outram', 'Rocher', 'Tanglin'. 
Those 'town_or_estate' that are not inside dfRegion will be inserted there under the appropriate region based on proximity to the current clustering. For those 'planning_areas' that do not have data from dfDwellUnits, they will be recorded as 0.

I need to check with URA and HDB to clarify this but it unlikely that I will receive the reply before project submission by 26th July 2021, which is only 3 working days from today. On receiving the reply, I will update the numbers appropriately.

Actions to take:<br>

**dfRegion:**
1. Add the newly created planning area 'Tengah', it's just above Choa Chu Kang
2. Add the newly created planning area 'Central Area', it's at area around Marina Bay Financial Centre (MBFC)

**dfDwellUnits:**
1. Rename 'Kallang/Whampoa' as Kallang. Kallang is a very old and small area, and it's probably merged into Kallang which is next to it and growing. Will reconfirm with URA/HDB.

In [25]:
# Actions for dfRegion
# -------------------
# 1. Add the newly created planning area 'Tengah', it's just above Choa Chu Kang
# 2. Add the newly created planning area 'Central Area', it's at area around Marina Bay Financial Centre (MBFC)

details = {
    'region': ['West Region','Central Region'],
    'planning_area': ['Tengah','Central Area']
}

dfTemp = pd.DataFrame(details)
dfRegions = dfRegions.append(dfTemp, ignore_index=True)
dfRegions = dfRegions.sort_values(by=['region','planning_area'])

# view and check
display(dfRegions[dfRegions.planning_area=='Tengah'])
display(dfRegions[dfRegions.planning_area=='Central Area'])

dfRegions.to_csv('dfRegions.csv',index=False)

,region,planning_area
29,West Region,Tengah


,region,planning_area
30,Central Region,Central Area


In [26]:
# Actions for dfDwellUnits:
# 1. Rename 'Kallang/Whampoa' as Kallang. 

dfDwellUnits.town_or_estate = dfDwellUnits.town_or_estate.apply(lambda x: 'Kallang' if x=='Kallang/Whampoa' else x)

temp = ['Kallang','Kallang/Whampoa']
for item in temp:
    print(f'Does {item} exist in dfDwellUnits ? {item in dfDwellUnits.town_or_estate.unique()}')

# rename columns
dfDwellUnits.rename(columns={
    "financial_year":'Year',
    "town_or_estate":'Town'
}, inplace=True)
    
dfDwellUnits.to_csv('dfDwellUnits.csv',index=False)

Does Kallang exist in dfDwellUnits ? True
Does Kallang/Whampoa exist in dfDwellUnits ? False


###  3.3 Clean and Transform Data: Census Data
output: df_HDB_ResidentAge

##### 3.3.1 Load and Clean Population Census 2000 data

In [27]:
# Load data, drop unnecessary info

colNames = ['Town','Total','00 to 04','05 to 09','10 to 14','15 to 19','20 to 24','25 to 29','30 to 34','35 to 39',
            '40 to 44','45 to 49','50 to 54','55 to 59','60 to 64','Over 65']

dfCensus2000 = pd.read_csv('Data/SingStat_Population/Geo_Distri/Census_Pop_2000/Table_1_Resi_Pop_by_DGP_Zone_Age_Grp/OutputFile.csv',
                          skiprows=7, index_col=False, names=colNames, header=None)

display(dfCensus2000.shape)
display(dfCensus2000.head(3))

(168, 16)

,Town,Total,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,Over 65
0,Ang Mo Kio,"180,634","9,296","11,732","11,158","12,142","14,326","15,994","13,835","15,478","16,062","15,909","13,952","8,904","7,495","14,351"
1,Cheng San,"32,029","1,506","1,984","1,854","2,260","2,701","2,862","2,603","2,678","2,803","2,968","2,563","1,567","1,304","2,376"
2,Chong Boon,"32,943","1,598","1,979","1,840","2,193","2,877","3,111","2,605","2,669","2,757","2,850","2,655","1,812","1,447","2,550"


In [28]:
# There are white spaces in the col 'Town', need to remove them.
dfCensus2000.Town = dfCensus2000.Town.apply(lambda x: x.strip())

In [29]:
# drop unnecessary columns
dfCensus2000.drop(columns='Total',inplace=True)
display(dfCensus2000.shape)
display(dfCensus2000.head(3))

(168, 15)

,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,Over 65
0,Ang Mo Kio,"9,296","11,732","11,158","12,142","14,326","15,994","13,835","15,478","16,062","15,909","13,952","8,904","7,495","14,351"
1,Cheng San,"1,506","1,984","1,854","2,260","2,701","2,862","2,603","2,678","2,803","2,968","2,563","1,567","1,304","2,376"
2,Chong Boon,"1,598","1,979","1,840","2,193","2,877","3,111","2,605","2,669","2,757","2,850","2,655","1,812","1,447","2,550"


In [30]:
# drop the unnecessary rows
print(dfCensus2000.shape)
display(dfCensus2000.tail(8))
print()
print('dropping the unneeded last 5 rows.')
dfCensus2000 = dfCensus2000.iloc[:-5,:]
print(dfCensus2000.shape)
display(dfCensus2000.tail(8))

(168, 15)


,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,Over 65
160,Others,"1,120","1,587","1,452","1,229","1,187","1,309","1,434","1,790","1,692","1,415","1,045",609,429,834
161,Others,268,338,314,288,374,419,426,501,482,418,454,286,219,612
162,Not Stated,275,410,354,286,311,435,407,483,529,464,387,245,180,510
163,SOURCE: SINGAPORE DEPARTMENT OF STATISTICS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,Data last updated: 01/04/2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,Generated by: SingStat Table Builder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,Date generated: 20/07/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,Contact: info@singstat.gov.sg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



dropping the unneeded last 5 rows.
(163, 15)


,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,Over 65
155,Yishun,"11,808","16,750","15,347","12,916","11,849","14,227","15,545","19,143","17,812","13,906","9,694","5,504","4,326","8,379"
156,Northland,"2,193","2,896","2,916","2,558","2,077","2,602","2,797","3,351","3,360","2,501","1,676",923,735,"1,381"
157,Yishun East,"2,421","3,710","2,692","1,887","1,929","2,386","3,010","3,818","3,046","2,159","1,433",819,597,"1,091"
158,Yishun South,"2,653","4,111","3,632","2,809","2,701","3,215","3,429","4,637","4,169","3,347","2,300","1,264","1,022","1,970"
159,Yishun West,"3,421","4,446","4,655","4,433","3,955","4,715","4,875","5,547","5,545","4,484","3,240","1,889","1,543","3,103"
160,Others,"1,120","1,587","1,452","1,229","1,187","1,309","1,434","1,790","1,692","1,415","1,045",609,429,834
161,Others,268,338,314,288,374,419,426,501,482,418,454,286,219,612
162,Not Stated,275,410,354,286,311,435,407,483,529,464,387,245,180,510


In [31]:
# Keep Only the towns that are in the dfRegions

# dfCensus2000.Town == dfRegions.planning_area

dfCensus2000['In_dfRegions'] = dfCensus2000.Town.apply(lambda x: True if x in dfRegions.planning_area.values else False)
dfCensus2000 = dfCensus2000[dfCensus2000['In_dfRegions']==True]
dfCensus2000.drop(columns='In_dfRegions',inplace=True)

display(dfCensus2000.head(3))
# dfCensus2000.to_csv('test.csv', index=False)

,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,Over 65
0,Ang Mo Kio,"9,296","11,732","11,158","12,142","14,326","15,994","13,835","15,478","16,062","15,909","13,952","8,904","7,495","14,351"
9,Bedok,"16,247","20,968","21,095","19,740","20,279","22,146","20,851","25,260","26,986","24,839","20,999","12,177","10,561","22,821"
17,Bukit Batok,"9,200","10,390","10,872","9,314","7,878","9,912","11,787","13,503","13,418","10,612","6,965","3,685","2,902","6,205"


In [32]:
# Check for Duplicates
dups =  dfCensus2000.Town.value_counts()
dups[dups>1]

Marine Parade    2
Kallang          2
Bukit Panjang    2
Name: Town, dtype: int64

In [33]:
# because the Main-Town ALWAYS appears before the sub-town, we can dedup by keeping the first occurance only.
# Moreover, in the downloaded dataset, the Main-town IS the Sum of all the sub-towns
# hence, we do not need to sum up main-town + sub-town
dfCensus2000.drop_duplicates(subset=['Town'], keep='first',inplace=True)

# change dtypes to appropriate types
### remove commas from numbers
### df = df.replace(',','', regex=True)
dfCensus2000[dfCensus2000.columns[1:]] = dfCensus2000[dfCensus2000.columns[1:]].replace(',','', regex=True)

# convert 'numeric' columns to numbers (all cols except 1st col)
### df[["a", "b"]] = df[["a", "b"]].apply(pd.to_numeric)
dfCensus2000[dfCensus2000.columns[1:]] = dfCensus2000[dfCensus2000.columns[1:]].apply(pd.to_numeric)

# Add a 'Year' column
dfCensus2000['Year'] = '2000'

# Reorder Columns
dfCensus2000 = dfCensus2000[['Year','Town', '00 to 04', '05 to 09', '10 to 14', '15 to 19', '20 to 24',
       '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', '50 to 54',
       '55 to 59', '60 to 64', 'Over 65']]

# Sort Towns and Save CSV
dfCensus2000 = dfCensus2000.sort_values(by='Town')
dfCensus2000 = dfCensus2000.reset_index(drop=True)

dfCensus2000.to_csv('Census2000_HDB_ResidentAge.csv',index=False)

##### 3.3.2 Load and Clean Population Census 2010 data

In [34]:
# Load data, drop unnecessary info

import pandas as pd

colNames = ['Town','Total','00 to 04','05 to 09','10 to 14','15 to 19','20 to 24','25 to 29','30 to 34','35 to 39',
            '40 to 44','45 to 49','50 to 54','55 to 59','60 to 64','Over 65']
                            
dfCensus2010 = pd.read_csv('Data/SingStat_Population/Geo_Distri/Census_Pop_2010/Table_1_Resi_Pop_by_Plan_Area_Age_Grp/OutputFile.csv',
                          skiprows=7, index_col=False, names=colNames, header=None)

display(dfCensus2010.shape)
display(dfCensus2010.head(3))

(233, 16)

,Town,Total,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,Over 65
0,Ang Mo Kio,"179,297","7,967","8,424","9,335","10,457","10,656","13,400","14,502","14,510","13,525","14,862","14,605","13,785","11,868","21,401"
1,Cheng San,"30,503","1,334","1,387","1,428","1,587","1,720","2,471","2,691","2,569","2,365","2,471","2,461","2,487","2,099","3,433"
2,Chong Boon,"29,903","1,254","1,304","1,429","1,563","1,713","2,348","2,469","2,326","2,241","2,431","2,388","2,380","2,137","3,920"


In [35]:
# There are white spaces in the col 'Town', need to remove them.
dfCensus2010.Town = dfCensus2010.Town.apply(lambda x: x.strip())

# drop unnecessary columns
dfCensus2010.drop(columns='Total',inplace=True)
display(dfCensus2010.shape)
display(dfCensus2010.head(3))

# drop the unnecessary rows
print(dfCensus2010.shape)
display(dfCensus2010.tail(8))
print()
print('dropping the unneeded rows.')
dfCensus2010 = dfCensus2010.iloc[:-6,:]
print(dfCensus2010.shape)
display(dfCensus2010.tail(12))
# dfCensus2010.to_csv('test.csv', index=False)

(233, 15)

,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,Over 65
0,Ang Mo Kio,"7,967","8,424","9,335","10,457","10,656","13,400","14,502","14,510","13,525","14,862","14,605","13,785","11,868","21,401"
1,Cheng San,"1,334","1,387","1,428","1,587","1,720","2,471","2,691","2,569","2,365","2,471","2,461","2,487","2,099","3,433"
2,Chong Boon,"1,254","1,304","1,429","1,563","1,713","2,348","2,469","2,326","2,241","2,431","2,388","2,380","2,137","3,920"


(233, 15)


,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,Over 65
225,Yishun West,"2,920","3,222","3,644","3,916","4,614","4,987","4,685","4,816","4,993","5,408","5,157","4,189","2,904","4,764"
226,Others,222,224,188,191,339,248,344,431,434,438,390,289,218,528
227,Notes:Planning areas refer to areas demarcated...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,SOURCE: SINGAPORE DEPARTMENT OF STATISTICS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229,Data last updated: 24/03/2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230,Generated by: SingStat Table Builder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,Date generated: 20/07/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,Contact: info@singstat.gov.sg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



dropping the unneeded rows.
(227, 15)


,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,Over 65
215,Woodlands South,"2,522","2,650","2,734","2,226","2,136","2,618","3,148","3,349","2,859","2,535","2,084","1,482",990,"1,365"
216,Woodlands West,"1,434","1,571","2,190","2,704","2,386","2,408","2,165","2,203","2,628","2,903","2,721","2,061","1,508","2,667"
217,Yishun,"8,985","10,149","11,856","14,249","14,775","14,516","14,097","14,525","14,859","17,292","15,949","12,566","8,653","12,743"
218,Khatib,492,516,688,939,970,931,855,841,886,"1,120","1,105",867,618,811
219,Lower Seletar,73,114,110,88,50,33,54,117,120,105,57,39,31,67
220,Northland,"1,643","1,867","2,163","2,511","2,667","2,574","2,603","2,564","2,655","3,055","2,887","2,130","1,419","1,999"
221,Springleaf,160,172,228,350,283,218,230,230,219,283,319,304,244,330
222,Yishun Central,67,45,102,161,144,147,116,79,109,161,152,124,76,125
223,Yishun East,"1,729","2,029","2,385","2,975","2,579","2,353","2,476","2,642","2,755","3,244","2,725","2,018","1,302","1,721"
224,Yishun South,"1,842","2,133","2,488","3,252","3,406","3,228","3,008","3,150","3,033","3,830","3,491","2,849","2,009","2,832"


In [36]:
# Keep Only the towns that are in the dfRegions
dfCensus2010['In_dfRegions'] = dfCensus2010.Town.apply(lambda x: True if x in dfRegions.planning_area.values else False)
dfCensus2010 = dfCensus2010[dfCensus2010['In_dfRegions']==True]
dfCensus2010.drop(columns='In_dfRegions',inplace=True)

dfCensus2010.head()
# dfCensus2010.to_csv('test.csv', index=False)

,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,Over 65
0,Ang Mo Kio,"7,967","8,424","9,335","10,457","10,656","13,400","14,502","14,510","13,525","14,862","14,605","13,785","11,868","21,401"
10,Bedok,"13,230","15,018","17,489","20,083","20,156","21,265","21,707","22,751","22,018","24,615","24,632","21,891","18,018","31,646"
19,Bishan,"3,941","4,759","5,691","7,188","6,338","5,930","6,010","7,354","7,102","8,041","8,245","6,725","5,141","8,833"
23,Bukit Batok,"7,187","8,516","9,738","10,427","10,625","11,332","10,941","11,829","12,259","13,049","12,433","9,911","6,487","9,464"
32,Bukit Merah,"8,049","6,892","6,894","7,479","7,940","10,865","13,871","13,495","11,796","11,650","12,057","11,352","10,782","24,000"


In [37]:
# Check for Duplicates
dups =  dfCensus2010.Town.value_counts()
dups[dups>1]

# because the Main-Town ALWAYS appears before the sub-town, we can dedup by keeping the first occurance only.
# Moreover, in the downloaded dataset, the Main-town IS the Sum of all the sub-towns
# hence, we do not need to sum up main-town + sub-town
dfCensus2010.drop_duplicates(subset=['Town'], keep='first',inplace=True)

# change dtypes to appropriate types
### remove commas from numbers
### df = df.replace(',','', regex=True)
dfCensus2010[dfCensus2010.columns[1:]] = dfCensus2010[dfCensus2010.columns[1:]].replace(',','', regex=True)

# convert 'numeric' columns to numbers (all cols except 1st col)
### df[["a", "b"]] = df[["a", "b"]].apply(pd.to_numeric)
dfCensus2010[dfCensus2010.columns[1:]] = dfCensus2010[dfCensus2010.columns[1:]].apply(pd.to_numeric)

# Add a 'Year' column
dfCensus2010['Year'] = '2010'

# Reorder Columns
dfCensus2010 = dfCensus2010[['Year','Town', '00 to 04', '05 to 09', '10 to 14', '15 to 19', '20 to 24',
       '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', '50 to 54',
       '55 to 59', '60 to 64', 'Over 65']]

# Sort Towns and Save CSV
dfCensus2010 = dfCensus2010.sort_values(by='Town')
dfCensus2010 = dfCensus2010.reset_index(drop=True)

dfCensus2010.to_csv('Census2010_HDB_ResidentAge.csv',index=False)

# notes: "Pungol" is a new town that does not appear in Census 2000

##### 3.3.3 Load and Clean General_HH_Survey 2015 data

In [38]:
# Load data, drop unnecessary info
import pandas as pd

colNames = ['Town','Total','00 to 04','05 to 09','10 to 14','15 to 19','20 to 24','25 to 29','30 to 34','35 to 39',
            '40 to 44','45 to 49','50 to 54','55 to 59','60 to 64','65 to 69', '70 to 74', '75 to 79', '80 to 84',
            'Over 85']
                            
dfCensus2015 = pd.read_csv('Data/SingStat_Population/Geo_Distri/General_HH_Survey_2015/T7_Resi_Pop_by_Plan_Area_Subzone_Age_Gender/OutputFile.csv',
                          skiprows=7, index_col=False, names=colNames, header=None)

display(dfCensus2015.shape)
display(dfCensus2015.head(3))

(393, 20)

,Town,Total,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,65 to 69,70 to 74,75 to 79,80 to 84,Over 85
0,Ang Mo Kio- Total,"174,770","6,790","7,660","8,290","9,320","10,310","11,170","12,250","13,070","13,710","13,000","14,010","13,800","12,980","11,050","6,670","5,140","3,250","2,300"
1,Ang Mo Kio Town Centre,"5,020",260,280,320,280,260,310,370,420,490,420,350,320,280,270,160,120,60,50
2,Cheng San,"29,770","1,290","1,180","1,290","1,400","1,570","1,830","2,490","2,490","2,460","2,220","2,320","2,290","2,320","1,920","1,070",790,480,370


In [39]:
# There are white spaces in the col 'Town', need to remove them.
dfCensus2015.Town = dfCensus2015.Town.apply(lambda x: str(x).strip())

# remove the '- Total' in 'Yishun- Total' etc
dfCensus2015.Town = dfCensus2015.Town.apply(lambda x: str(x).replace('- Total','') )

# drop unnecessary columns
dfCensus2015.drop(columns='Total',inplace=True)
display(dfCensus2015.shape)
display(dfCensus2015.head(3))

# drop the unnecessary rows
print(dfCensus2015.shape)
display(dfCensus2015.tail(8))
print()
print('dropping the unneeded rows.')
dfCensus2015 = dfCensus2015.iloc[:-15,:]
print(dfCensus2015.shape)
display(dfCensus2015.tail(12))

# dfCensus2015.to_csv('test.csv', index=False)

(393, 19)

,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,65 to 69,70 to 74,75 to 79,80 to 84,Over 85
0,Ang Mo Kio,"6,790","7,660","8,290","9,320","10,310","11,170","12,250","13,070","13,710","13,000","14,010","13,800","12,980","11,050","6,670","5,140","3,250","2,300"
1,Ang Mo Kio Town Centre,260,280,320,280,260,310,370,420,490,420,350,320,280,270,160,120,60,50
2,Cheng San,"1,290","1,180","1,290","1,400","1,570","1,830","2,490","2,490","2,460","2,220","2,320","2,290","2,320","1,920","1,070",790,480,370


(393, 19)


,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,65 to 69,70 to 74,75 to 79,80 to 84,Over 85
385,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
386,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,Data last updated: 09/03/2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390,Generated by: SingStat Table Builder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
391,Date generated: 20/07/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392,Contact: info@singstat.gov.sg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



dropping the unneeded rows.
(378, 19)


,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,65 to 69,70 to 74,75 to 79,80 to 84,Over 85
366,Woodlands South,"2,230","2,550","2,630","2,780","2,310","2,490","3,070","3,190","3,200","2,860","2,540","2,120","1,600","1,070",530,350,220,140
367,Woodlands West,"1,370","1,450","1,690","2,350","2,950","2,450","2,320","1,950","2,220","2,670","2,960","2,790","2,070","1,470",860,690,400,240
368,Yishun,"10,230","9,950","10,440","12,190","14,650","17,010","17,040","15,420","15,110","14,950","17,360","15,950","12,310","8,340","4,430","3,190","1,940","1,480"
369,Khatib,340,440,490,640,920,900,780,720,770,810,"1,060","1,030",770,540,280,170,120,80
370,Lower Seletar,290,210,210,180,140,300,340,230,260,240,190,110,60,50,30,20,10,10
371,Nee Soon,50,40,50,50,50,60,40,70,80,80,80,60,50,50,40,20,20,10
372,Northland,"1,110","1,460","1,630","1,970","2,380","2,490","2,220","2,270","2,270","2,340","2,800","2,690","1,910","1,270",640,460,260,220
373,Springleaf,140,170,220,260,350,290,190,250,250,250,310,330,300,250,130,90,40,50
374,Yishun Central,50,70,50,110,150,120,120,80,70,110,150,140,120,70,40,20,10,10
375,Yishun East,"4,580","3,210","2,850","3,110","3,600","5,010","6,470","4,800","3,890","3,660","4,060","3,340","2,570","1,720",820,590,360,230


In [40]:
# replace ' - ' with 0
lst = dfCensus2015.columns[1:]
for col in lst:
    dfCensus2015[col] = dfCensus2015[col].apply(lambda x: '0' if x.strip() == '-' else x)

# replace Nulls with 0, coz adding a number to a NaN results in a NaN
dfCensus2015.fillna(0,inplace=True)

#dfCensus2015.to_csv('test.csv', index=False)

In [41]:
# Keep Only the towns that are in the dfRegions
dfCensus2015['In_dfRegions'] = dfCensus2015.Town.apply(lambda x: True if x in dfRegions.planning_area.values else False)
dfCensus2015 = dfCensus2015[dfCensus2015['In_dfRegions']==True]
dfCensus2015.drop(columns='In_dfRegions',inplace=True)

dfCensus2015.head()
#dfCensus2015.to_csv('test.csv', index=False)

,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,65 to 69,70 to 74,75 to 79,80 to 84,Over 85
0,Ang Mo Kio,"6,790","7,660","8,290","9,320","10,310","11,170","12,250","13,070","13,710","13,000","14,010","13,800","12,980","11,050","6,670","5,140","3,250","2,300"
13,Bedok,"11,690","13,400","14,750","16,930","19,450","19,860","19,270","20,850","22,520","21,460","23,430","23,380","20,590","16,750","9,310","7,330","4,760","4,010"
22,Bishan,"3,430","4,330","4,710","5,520","6,860","6,460","5,720","6,000","7,070","6,800","7,540","7,700","6,360","4,860","2,730","2,140","1,370","1,090"
31,Bukit Batok,"5,510","6,890","7,970","9,240","10,070","10,460","9,760","9,940","10,980","11,310","12,190","11,770","9,370","5,990","3,060","2,230","1,450","1,090"
41,Bukit Merah,"7,210","7,480","6,640","6,860","7,700","9,340","11,070","12,840","12,730","11,320","11,320","11,740","10,920","10,030","6,480","5,590","3,540","3,010"


In [42]:
# Check for Duplicates
dups =  dfCensus2015.Town.value_counts()
dups[dups>1]

# because the Main-Town ALWAYS appears before the sub-town, we can dedup by keeping the first occurance only.
# Moreover, in the downloaded dataset, the Main-town IS the Sum of all the sub-towns
# hence, we do not need to sum up main-town + sub-town
dfCensus2015.drop_duplicates(subset=['Town'], keep='first',inplace=True)

# change dtypes to appropriate types
### remove commas from numbers
### df = df.replace(',','', regex=True)
dfCensus2015[dfCensus2015.columns[1:]] = dfCensus2015[dfCensus2015.columns[1:]].replace(',','', regex=True)

# convert 'numeric' columns to numbers (all cols except 1st col)
### df[["a", "b"]] = df[["a", "b"]].apply(pd.to_numeric)
dfCensus2015[dfCensus2015.columns[1:]] = dfCensus2015[dfCensus2015.columns[1:]].apply(pd.to_numeric)

# consolidate age '65 and above' into one column to standardised with dfCensusNNNN prior to this one
# because previous census did not provide any data in age bins that are over 65.
dfCensus2015['Over 65'] = dfCensus2015['65 to 69'] + dfCensus2015['70 to 74'] + dfCensus2015['75 to 79']+\
dfCensus2015['80 to 84'] + dfCensus2015['Over 85']

# drop unused columns
dfCensus2015.drop(columns=['65 to 69', '70 to 74', '75 to 79', '80 to 84','Over 85'], inplace=True)

# Add a 'Year' column
dfCensus2015['Year'] = '2015'

# Reorder Columns
dfCensus2015 = dfCensus2015[['Year','Town', '00 to 04', '05 to 09', '10 to 14', '15 to 19', '20 to 24',
       '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', '50 to 54',
       '55 to 59', '60 to 64', 'Over 65']]

# Sort Towns and Save CSV
dfCensus2015 = dfCensus2015.sort_values(by='Town')
dfCensus2015 = dfCensus2015.reset_index(drop=True)

#dfCensus2015.to_csv('test.csv', index=False)
dfCensus2015.to_csv('Census2015_HDB_ResidentAge.csv',index=False)

##### 3.3.4 Load and Clean Population Census 2020 data

In [43]:
# Load data, drop unnecessary info
import pandas as pd

colNames = ['Town','Total','00 to 04','05 to 09','10 to 14','15 to 19','20 to 24','25 to 29','30 to 34','35 to 39',
            '40 to 44','45 to 49','50 to 54','55 to 59','60 to 64','65 to 69', '70 to 74', '75 to 79', '80 to 84',
            '85-89','Over 90']
   
   
dfCensus2020 = pd.read_csv('Data/SingStat_Population/Geo_Distri/Census_Pop_2020/Table_88_Resi_Pop_by_Plan_Area_Age_Grp_Gender/OutputFile.csv',
                          skiprows=7, index_col=False, names=colNames, header=None)

display(dfCensus2020.shape)
display(dfCensus2020.head(3))

(393, 21)

,Town,Total,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,...,45 to 49,50 to 54,55 to 59,60 to 64,65 to 69,70 to 74,75 to 79,80 to 84,85-89,Over 90
0,Ang Mo Kio - Total,"162,280","5,280","6,100","7,030","7,600","8,680","10,320","10,490","10,420",...,"12,410","11,860","12,780","12,730","11,960","9,930","5,770","4,150","2,280","1,130"
1,Ang Mo Kio Town Centre,"4,810",170,240,280,320,270,280,290,330,...,470,370,320,300,250,230,140,100,40,20
2,Cheng San,"28,070","1,060","1,040","1,040","1,160","1,330","1,710","2,000","2,150",...,"2,200","2,050","2,120","2,120","2,170","1,730",960,640,350,180


In [44]:
# remove the '- Total' in 'Yishun- Total' etc
dfCensus2020.Town = dfCensus2020.Town.apply(lambda x: str(x).replace('- Total','') )

# There are white spaces in the col 'Town', need to remove them.
dfCensus2020.Town = dfCensus2020.Town.apply(lambda x: str(x).strip())

# drop unnecessary columns
dfCensus2020.drop(columns='Total',inplace=True)
display(dfCensus2020.shape)
display(dfCensus2020.head(3))

# drop the unnecessary rows
print(dfCensus2020.shape)
display(dfCensus2020.tail(8))
print()
print('dropping the unneeded rows.')
dfCensus2020 = dfCensus2020.iloc[:-15,:]
print(dfCensus2020.shape)
display(dfCensus2020.tail(12))

#dfCensus2020.to_csv('test.csv', index=False)

(393, 20)

,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,65 to 69,70 to 74,75 to 79,80 to 84,85-89,Over 90
0,Ang Mo Kio,"5,280","6,100","7,030","7,600","8,680","10,320","10,490","10,420","11,350","12,410","11,860","12,780","12,730","11,960","9,930","5,770","4,150","2,280","1,130"
1,Ang Mo Kio Town Centre,170,240,280,320,270,280,290,330,400,470,370,320,300,250,230,140,100,40,20
2,Cheng San,"1,060","1,040","1,040","1,160","1,330","1,710","2,000","2,150","2,070","2,200","2,050","2,120","2,120","2,170","1,730",960,640,350,180


(393, 20)


,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,65 to 69,70 to 74,75 to 79,80 to 84,85-89,Over 90
385,Yishun South,"1,930","1,690","1,920","2,090","2,500","3,600","3,720","2,940","2,810","3,010","2,890","3,690","3,250","2,570","1,740",830,580,320,170
386,Yishun West,"1,780","1,990","2,440","2,790","3,410","3,810","3,590","3,600","3,810","4,230","4,330","4,840","4,500","3,580","2,320","1,280",880,480,240
387,Note: Planning areas refer to areas demarcate...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,SOURCE: SINGAPORE DEPARTMENT OF STATISTICS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389,Data last updated: 18/06/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390,Generated by: SingStat Table Builder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
391,Date generated: 20/07/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392,Contact: info@singstat.gov.sg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



dropping the unneeded rows.
(378, 20)


,Town,00 to 04,05 to 09,10 to 14,15 to 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,45 to 49,50 to 54,55 to 59,60 to 64,65 to 69,70 to 74,75 to 79,80 to 84,85-89,Over 90
366,Murai,30,50,20,20,130,50,30,60,70,40,30,50,50,10,-,-,-,-,-
367,Woodlands,"11,490","12,540","14,460","17,110","21,430","20,740","18,160","17,730","17,910","20,320","21,560","20,200","15,890","10,830","6,980","3,530","2,410","1,260",580
368,Greenwood Park,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
369,Midview,"1,590","1,640","1,870","2,480","3,460","3,220","2,480","2,290","2,100","2,520","3,270","2,960","2,100","1,400",830,470,320,160,110
370,North Coast,300,350,470,650,970,"1,000",700,640,700,850,"1,030","1,210","1,030",800,670,340,230,120,50
371,Senoko West,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
372,Woodgrove,980,"1,330","1,740","2,280","3,320","3,210","2,060","1,930","2,170","2,610","3,300","3,000","2,350","1,500",920,480,340,190,110
373,Woodlands East,"4,960","5,280","6,010","6,880","8,020","7,660","7,520","7,270","7,460","8,490","8,020","7,510","5,710","3,680","2,220","1,030",720,380,170
374,Woodlands Regional Centre,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
375,Woodlands South,"2,100","2,620","2,850","2,990","3,080","2,630","2,880","3,420","3,540","3,590","3,190","2,600","2,020","1,510","1,000",480,290,170,60


In [45]:
# replace ' - ' with 0
lst = dfCensus2020.columns[1:]
for col in lst:
    dfCensus2020[col] = dfCensus2020[col].apply(lambda x: '0' if x.strip() == '-' else x)

# replace Nulls with 0, coz adding a number to a NaN results in a NaN
dfCensus2020.fillna(0,inplace=True)

# dfCensus2020.to_csv('test.csv', index=False)

# Keep Only the towns that are in the dfRegions
dfCensus2020['In_dfRegions'] = dfCensus2020.Town.apply(lambda x: True if x in dfRegions.planning_area.values else False)
dfCensus2020 = dfCensus2020[dfCensus2020['In_dfRegions']==True]
dfCensus2020.drop(columns='In_dfRegions',inplace=True)

# dfCensus2020.head()
#dfCensus2020.to_csv('test.csv', index=False)

# Check for Duplicates
dups =  dfCensus2020.Town.value_counts()
dups[dups>1]

# because the Main-Town ALWAYS appears before the sub-town, we can dedup by keeping the first occurance only.
# Moreover, in the downloaded dataset, the Main-town IS the Sum of all the sub-towns
# hence, we do not need to sum up main-town + sub-town
dfCensus2020.drop_duplicates(subset=['Town'], keep='first',inplace=True)

# change dtypes to appropriate types
### remove commas from numbers
### df = df.replace(',','', regex=True)
dfCensus2020[dfCensus2020.columns[1:]] = dfCensus2020[dfCensus2020.columns[1:]].replace(',','', regex=True)

# convert 'numeric' columns to numbers (all cols except 1st col)
### df[["a", "b"]] = df[["a", "b"]].apply(pd.to_numeric)
dfCensus2020[dfCensus2020.columns[1:]] = dfCensus2020[dfCensus2020.columns[1:]].apply(pd.to_numeric)

#dfCensus2020.to_csv('test.csv', index=False)

In [46]:
# consolidate age '65 and above' into one column to standardised with dfCensusNNNN prior to this one
# because previous census did not provide any data in age bins that are over 65.
dfCensus2020['Over 65'] = dfCensus2020['65 to 69'] + dfCensus2020['70 to 74'] + dfCensus2020['75 to 79']+\
dfCensus2020['80 to 84'] + dfCensus2020['85-89'] + dfCensus2020['Over 90']

# drop unused columns
dfCensus2020.drop(columns=['65 to 69', '70 to 74', '75 to 79', '80 to 84','85-89','Over 90'], inplace=True)

# Add a 'Year' column
dfCensus2020['Year'] = '2020'

# Reorder Columns
dfCensus2020 = dfCensus2020[['Year','Town', '00 to 04', '05 to 09', '10 to 14', '15 to 19', '20 to 24',
       '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', '50 to 54',
       '55 to 59', '60 to 64', 'Over 65']]

# Sort Towns and Save CSV
dfCensus2020 = dfCensus2020.sort_values(by='Town')
dfCensus2020 = dfCensus2020.reset_index(drop=True)

#dfCensus2020.to_csv('test.csv', index=False)
dfCensus2020.to_csv('Census2020_HDB_ResidentAge.csv',index=False)

In [47]:
dfCensus2020.columns

Index(['Year', 'Town', '00 to 04', '05 to 09', '10 to 14', '15 to 19',
       '20 to 24', '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49',
       '50 to 54', '55 to 59', '60 to 64', 'Over 65'],
      dtype='object')

##### 3.3.5 Consolidate All the Census data into a Master Census Dataframe

###### 3.3.5.1 Consolidate Census2000 and Census2010

In [48]:
df_Diff = dfCensus2010[dfCensus2010.columns[2:]] - dfCensus2000[dfCensus2000.columns[2:]]
df_ChgPerYear = df_Diff / (2010-2000)  # diff between 2010 and 2000 is linearly interpolated


df2001 = dfCensus2000.copy()
df2001.Year = '2001'
df2001[df2001.columns[2:]] = dfCensus2000[dfCensus2000.columns[2:]] + df_ChgPerYear

df2002 = df2001.copy()
df2002.Year = '2002'
df2002[df2002.columns[2:]] = df2001[df2001.columns[2:]] + df_ChgPerYear

df2003 = df2002.copy()
df2003.Year = '2003'
df2003[df2003.columns[2:]] = df2002[df2002.columns[2:]] + df_ChgPerYear

df2004= df2003.copy()
df2004.Year = '2004'
df2004[df2004.columns[2:]] = df2003[df2003.columns[2:]] + df_ChgPerYear

df2005= df2004.copy()
df2005.Year = '2005'
df2005[df2005.columns[2:]] = df2004[df2004.columns[2:]] + df_ChgPerYear

df2006= df2005.copy()
df2006.Year = '2006'
df2006[df2006.columns[2:]] = df2005[df2005.columns[2:]] + df_ChgPerYear

df2007= df2006.copy()
df2007.Year = '2007'
df2007[df2007.columns[2:]] = df2006[df2006.columns[2:]] + df_ChgPerYear

df2008= df2007.copy()
df2008.Year = '2008'
df2008[df2008.columns[2:]] = df2007[df2007.columns[2:]] + df_ChgPerYear

df2009= df2008.copy()
df2009.Year = '2009'
df2009[df2009.columns[2:]] = df2008[df2008.columns[2:]] + df_ChgPerYear

df2010 = dfCensus2010.copy()

df_HDB_ResidentAge = pd.concat([dfCensus2000,df2001,df2002,df2003,df2004,df2005,df2006,df2007,df2008,df2009,dfCensus2010])

# convert to int
df_HDB_ResidentAge[df_HDB_ResidentAge.columns[2:]] = \
df_HDB_ResidentAge[df_HDB_ResidentAge.columns[2:]].astype('int64')

# df_HDB_ResidentAge.to_csv('test.csv', index=False)

###### 3.3.5.2  Add Census2015

In [49]:
df_Diff = dfCensus2015[dfCensus2015.columns[2:]] - dfCensus2010[dfCensus2010.columns[2:]]
df_ChgPerYear = df_Diff / (2015-2010)  # diff between the two time period is linearly interpolated


df2011 = dfCensus2010.copy()
df2011.Year = '2011'
df2011[df2011.columns[2:]] = dfCensus2010[dfCensus2010.columns[2:]] + df_ChgPerYear

df2012 = df2011.copy()
df2012.Year = '2012'
df2012[df2012.columns[2:]] = df2011[df2011.columns[2:]] + df_ChgPerYear

df2013 = df2012.copy()
df2013.Year = '2013'
df2013[df2013.columns[2:]] = df2012[df2012.columns[2:]] + df_ChgPerYear

df2014= df2013.copy()
df2014.Year = '2014'
df2014[df2014.columns[2:]] = df2013[df2013.columns[2:]] + df_ChgPerYear

df2015= dfCensus2015.copy()

dfTemp = pd.concat([df2011,df2012,df2013,df2014,dfCensus2015])

# convert to int
dfTemp[dfTemp.columns[2:]] = \
dfTemp[dfTemp.columns[2:]].astype('int64')

# add to main df
df_HDB_ResidentAge = pd.concat([df_HDB_ResidentAge, dfTemp])

# df_HDB_ResidentAge.to_csv('test.csv', index=False)

###### 3.3.5.3  Add Census2020

In [50]:
df_Diff = dfCensus2020[dfCensus2020.columns[2:]] - dfCensus2015[dfCensus2015.columns[2:]]
df_ChgPerYear = df_Diff / (2020-2015)  # diff between the two time period is linearly interpolated


df2016 = dfCensus2015.copy()
df2016.Year = '2016'
df2016[df2016.columns[2:]] = dfCensus2015[dfCensus2015.columns[2:]] + df_ChgPerYear

df2017 = df2016.copy()
df2017.Year = '2017'
df2017[df2017.columns[2:]] = df2016[df2016.columns[2:]] + df_ChgPerYear

df2018 = df2017.copy()
df2018.Year = '2018'
df2018[df2018.columns[2:]] = df2017[df2017.columns[2:]] + df_ChgPerYear

df2019= df2018.copy()
df2019.Year = '2019'
df2019[df2019.columns[2:]] = df2018[df2018.columns[2:]] + df_ChgPerYear

df2020= dfCensus2020.copy()

dfTemp = pd.concat([df2016,df2017,df2018,df2019,dfCensus2020])

# convert to int
dfTemp[dfTemp.columns[2:]] = \
dfTemp[dfTemp.columns[2:]].astype('int64')

# add to main df
df_HDB_ResidentAge = pd.concat([df_HDB_ResidentAge, dfTemp])



In [51]:
# melt the df

idCols = ['Year','Town']
valueCols = df_HDB_ResidentAge.columns.difference(idCols)

df_HDB_ResidentAge = pd.melt(df_HDB_ResidentAge, 
                            id_vars=idCols,
                            value_vars=valueCols,
                            var_name = 'Age Grp',
                            value_name = 'Num_Residents')

# dfDwellUnits = pd.melt(dfDwellUnits, id_vars=['financial_year','town_or_estate'],
#         value_vars=['1rm_2rm','3rm','4rm','5rm_Exec'],
#         var_name='Dwelling_Type', value_name='num_DwellUnits')


# save to csv
df_HDB_ResidentAge.to_csv('dfHDB_ResidentAge.csv', index=False)

###  3.4 Clean and Transform Data: Electricity Consumption in HDB
output: df_HDB_ElectricityCons

In [52]:
# take note: it is Ave mth elec cons in kwh

df_HDB_ElecCons = pd.read_csv('Data/EMA/3_6_Ave_Ann_HH_Elec_Cons_by_Plan_Area_Dwell_Type/OutputFile.csv')

display(df_HDB_ElecCons.shape)
display(df_HDB_ElecCons.dtypes)
display(df_HDB_ElecCons.head(3))
print()
display(df_HDB_ElecCons.isnull().sum())

(2175, 5)

year                                            int64
planning_area                                  object
region                                         object
dwelling_type                                  object
average_monthly_electricity_consumption_kwh    object
dtype: object

,year,planning_area,region,dwelling_type,average_monthly_electricity_consumption_kwh
0,2019,Bishan,Central Region,1-room / 2-room,142.7
1,2019,Bukit Merah,Central Region,1-room / 2-room,158
2,2019,Bukit Timah,Central Region,1-room / 2-room,150.3


year                                           0
planning_area                                  0
region                                         0
dwelling_type                                  0
average_monthly_electricity_consumption_kwh    0
dtype: int64

In [53]:
#rename columns
df_HDB_ElecCons.rename(columns = {
    'year': 'Year', 
    'planning_area': 'Town',
    'region':'Region',
    'dwelling_type': 'Dwell_Type',
    'average_monthly_electricity_consumption_kwh':'AveMthly_kwh'    
}, inplace=True)

# reorg columns
df_HDB_ElecCons = df_HDB_ElecCons[['Year','Region','Town','Dwell_Type','AveMthly_kwh']]
df_HDB_ElecCons.head(2)

,Year,Region,Town,Dwell_Type,AveMthly_kwh
0,2019,Central Region,Bishan,1-room / 2-room,142.7
1,2019,Central Region,Bukit Merah,1-room / 2-room,158


In [54]:
# delete rows where 'Dwell_Type' is not 'Overall'
df_HDB_ElecCons = df_HDB_ElecCons[df_HDB_ElecCons.Dwell_Type != 'Overall']
#df_HDB_ElecCons.to_csv('test.csv', index=False)

In [55]:
df_HDB_ElecCons.Dwell_Type.unique()

array(['1-room / 2-room', '3-room', '4-room', '5-room / Executive'],
      dtype=object)

In [56]:
# rename Dwell_Type to
## ['1rm_2rm','3rm','4rm','5rm_Exec']
df_HDB_ElecCons.Dwell_Type = df_HDB_ElecCons.Dwell_Type.apply(lambda x: '1rm_2rm' if x=='1-room / 2-room' else x)
df_HDB_ElecCons.Dwell_Type = df_HDB_ElecCons.Dwell_Type.apply(lambda x: '3rm' if x=='3-room' else x)
df_HDB_ElecCons.Dwell_Type = df_HDB_ElecCons.Dwell_Type.apply(lambda x: '4rm' if x=='4-room' else x)
df_HDB_ElecCons.Dwell_Type = df_HDB_ElecCons.Dwell_Type.apply(lambda x: '5rm_Exec' if x=='5-room / Executive' else x)

df_HDB_ElecCons.Dwell_Type.unique()
# df_HDB_ElecCons.to_csv('test.csv', index=False)

array(['1rm_2rm', '3rm', '4rm', '5rm_Exec'], dtype=object)

In [57]:
# check and correct dtypes if req
df_HDB_ElecCons.dtypes

Year             int64
Region          object
Town            object
Dwell_Type      object
AveMthly_kwh    object
dtype: object

In [58]:
df_HDB_ElecCons.AveMthly_kwh = df_HDB_ElecCons.AveMthly_kwh.apply(lambda x: str(x).replace('-','0') )
df_HDB_ElecCons.AveMthly_kwh = df_HDB_ElecCons.AveMthly_kwh.astype('float64')

df_HDB_ElecCons.dtypes

Year              int64
Region           object
Town             object
Dwell_Type       object
AveMthly_kwh    float64
dtype: object

In [59]:
# add col 
df_HDB_ElecCons['Total_kwh'] = df_HDB_ElecCons.AveMthly_kwh * 12

# sort by 'Year','Region', 'Town', 'Dwell_Type'
df_HDB_ElecCons = df_HDB_ElecCons.sort_values(by=['Year','Region', 'Town', 'Dwell_Type'])

# save csv
df_HDB_ElecCons.to_csv('dfHDB_ElecCons.csv', index=False)